In [1]:
# Import lib
# ===========================================================
import csv
import pandas as pd
import numpy as np
import random
import time
import collections
import math
import sys
from tqdm import tqdm
from time import sleep

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from datascience import *
from scipy import stats

import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [2]:
# Initialize useful data
# ===========================================================
# with open('clinvar_conflicting_clean.csv', 'r') as f:
#     reader = csv.reader(f)
#     temp_rows = list(reader)
df = pd.read_csv('clinvar_conflicting_clean.csv', low_memory=False)
columns_to_change = ['ORIGIN', 'EXON', 'INTRON', 'STRAND', 'LoFtool', 'CADD_PHRED', 'CADD_RAW', 'BLOSUM62']
df[['CLNVI', 'MC', 'SYMBOL', 'Feature_type', 'Feature', 'BIOTYPE', 
 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 
 'BAM_EDIT', 'SIFT', 'PolyPhen']] = df[['CLNVI', 'MC', 'SYMBOL', 'Feature_type', 'Feature', 'BIOTYPE', 
 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 
 'BAM_EDIT', 'SIFT', 'PolyPhen']].fillna(value="null")

In [3]:
# map categorical data to numerical data
# ===========================================================
def uniq_val(column):
    input_domain = set([column[i][0] for i in range(len(column))])
    return input_domain

def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

def map_categ2numer():
    for attribute in df.columns.values:
        
        if attribute == 'CLASS': continue
        # if it's numerical data, normalize it in position
        if is_numeric(df[[attribute]].values[0]):
            # extract the max of the abs(data), take as magnitude
            mag = max(max(df[[attribute]].values), abs(min(df[[attribute]].values)))
            df[[attribute]] = df[[attribute]].div(mag)
        else:  # if it's categorical data, map it according to its index and normalize it
            values_of_this_attrib = uniq_val(df[[attribute]].values)
            length = len(values_of_this_attrib)
            i = 0
            for value in values_of_this_attrib:
    #             print(values_of_this_attrib[i])
#                 df[[attribute]] = df[[attribute]].replace(value, i / length)
                df[[attribute]] = df[[attribute]].replace(np.random.rand())
                i += 1

map_categ2numer()
df.head()

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDN,CLNHGVS,...,Codons,STRAND,BAM_EDIT,SIFT,PolyPhen,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62,CLASS
0,1,955563,G,C,0.0000,0.00000,0.0000,"MedGen:C3808739,OMIM:615120|MedGen:CN169374","Myasthenic_syndrome,_congenital,_8|not_specified",NC_000001.10:g.955563G>C,...,cGg/cCg,1.0,null,null,null,0.421,11.390,1.133255,-2.0,0
1,1,955597,G,T,0.0000,0.42418,0.2826,MedGen:CN169374,not_specified,NC_000001.10:g.955597G>T,...,ccG/ccT,1.0,null,null,null,0.421,8.150,0.599088,NaN,0
2,1,955619,G,C,0.0000,0.03475,0.0088,"MedGen:C3808739,OMIM:615120|MedGen:CN169374","Myasthenic_syndrome,_congenital,_8|not_specified",NC_000001.10:g.955619G>C,...,Gtc/Ctc,1.0,null,null,null,0.421,3.288,0.069819,1.0,1
3,1,957640,C,T,0.0318,0.02016,0.0328,"MedGen:C3808739,OMIM:615120|MedGen:CN169374","Myasthenic_syndrome,_congenital,_8|not_specified",NC_000001.10:g.957640C>T,...,gaC/gaT,1.0,null,null,null,0.421,12.560,1.356499,NaN,0
4,1,976059,C,T,0.0000,0.00022,0.0010,MedGen:CN169374,not_specified,NC_000001.10:g.976059C>T,...,Ctg/Ttg,1.0,null,null,null,0.421,17.740,2.234711,NaN,1


In [4]:
df.to_csv('clinvar_conflicting_mapped_random.csv', index=False)

In [5]:
# df = pd.read_csv('clinvar_conflicting_mapped.csv', low_memory=False)

In [6]:
# min(df[['POS']].values)

In [7]:
# np.random.rand()